In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import kerastuner as kt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt
import pprint

#  Import and read the heart_disease_dataset.csv.
import pandas as pd

url= 'https://raw.githubusercontent.com/sallyca530/Fetal_Health/main/data/fetal_health.csv'
fetal_health_df = pd.read_csv(url)
fetal_health_df.head()

<ipython-input-3-d704999d716c>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


In [4]:
#confirming lenght to make sure no data was lost
len(fetal_health_df)

2126

In [5]:
# nuniuqe to view the unique data in each column
fetal_health_df.nunique()

baseline value                                             48
accelerations                                              20
fetal_movement                                            102
uterine_contractions                                       16
light_decelerations                                        16
severe_decelerations                                        2
prolongued_decelerations                                    6
abnormal_short_term_variability                            75
mean_value_of_short_term_variability                       57
percentage_of_time_with_abnormal_long_term_variability     87
mean_value_of_long_term_variability                       249
histogram_width                                           154
histogram_min                                             109
histogram_max                                              86
histogram_number_of_peaks                                  18
histogram_number_of_zeroes                                  9
histogra

In [6]:
# Split our preprocessed data into our features and target arrays
y = fetal_health_df['fetal_health']
X = fetal_health_df.drop(['fetal_health'], axis = 1)


In [7]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(1594, 21)
(532, 21)


In [10]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=21))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [11]:
# Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [12]:
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_train_scaled data type: {X_train_scaled.dtype}")
print(f"y_train data type: {y_train.dtype}")

X_train_scaled shape: (1594, 21)
y_train shape: (1594,)
X_train_scaled data type: float64
y_train data type: float64


In [13]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))


Trial 60 Complete [00h 00m 12s]
val_accuracy: 0.7781955003738403

Best val_accuracy So Far: 0.7781955003738403
Total elapsed time: 00h 05m 36s


In [14]:
test = tuner.get_best_hyperparameters(5)

In [15]:
models = tuner.get_best_models(5)[0]

In [16]:
#top hyperparameters
test[0].values

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 3,
 'units_0': 5,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0,
 'units_1': 1,
 'units_2': 1}

In [17]:
models

In [18]:
for v in test:
  pprint.pprint(v.values)

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 3,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 5,
 'units_1': 1,
 'units_2': 1}
{'activation': 'relu',
 'first_units': 5,
 'num_layers': 5,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 1,
 'units_1': 3,
 'units_2': 7,
 'units_3': 1,
 'units_4': 1}
{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 3,
 'units_1': 5,
 'units_2': 7,
 'units_3': 7,
 'units_4': 3}
{'activation': 'sigmoid',
 'first_units': 7,
 'num_layers': 1,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 1,
 'units_1': 7,
 'units_2': 9,
 'units_3': 7,
 'units_4': 1}
{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 6,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tun

In [19]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 3,
 'units_0': 5,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0,
 'units_1': 1,
 'units_2': 1}

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = models.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

17/17 - 1s - loss: 0.0000e+00 - accuracy: 0.7782 - 731ms/epoch - 43ms/step
Loss: 0.0, Accuracy: 0.7781955003738403
